# Titanic competition with TensorFlow Decision Forests

## Import dependencies

In [1]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

# If haven't installed the required packages, uncomment and run:
# (若尚未安裝以下套件，請先執行:)
# !pip install tensorflow tensorflow_decision_forests

Found TF-DF 1.12.0


## Load dataset

In [2]:
# 讀取 Titanic 的訓練與測試資料集
train_df = pd.read_csv("../data/train.csv")
serving_df = pd.read_csv("../data/test.csv")

# 檢視前 10 筆資料
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Prepare dataset

In [3]:
def preprocess(df):     # 定義函式 preprocess，目的是對Titanic資料做資料清理與欄位擴充處理。
    df = df.copy()      # 複製傳進來的 DataFrame，避免直接修改原始資料 (good practice)。

# Name欄位處理: 清除雜訊符號
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
# Ticket欄位處理: 拆成兩部分
# 第一部分: ticket_number(x)取最後一段(通常是號碼)
    def ticket_number(x):
        return x.split()[-1]
    
#第二部分: ticket_item(x)取前面的識別代碼部分
    def ticket_item(x):
        items = x.split()
        if len(items) == 1:
            return "NONE"
        return "_".join(items[:-1])

# .apply: Apply應用這些函式到 DataFrame 欄位:  
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
# 回傳處理好的資料 (dataframe = df)    
    return df

#把 train_df 和 serving_df 各自傳進去 preprocess()函式，做完資料清理後得到兩份新的資料集。
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

# 顯示前 5 筆資料:
preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [4]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


#### `.columns`是什麼意思?
`.columns`是Pandas`DataFrame`的一個屬性，它會回傳這個資料表的**所有欄位名稱(column names)**，也就是CSV檔案的「表頭」。

## Convert Pandas dataset to TensorFlow Dataset

In [5]:
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] = tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

### 第一段: 定義函數`tokenize_names`

- `def tokenize_names(...)`  
  這是定義一個函數(function)叫做`tokenize_names`，  
  它的用途是: **將每一筆資料的Name欄位進行文字分詞(tokenize)**。  
- `features`  
  這是輸入的特徵(欄位資料)，在TensorFlow Dataset中，一筆資料會長得像一個字典(dictionary)，例如:  
  ```
  features = {
      "Name": "Braund, Mr. Owen Harris",
      "Age": 22,
      "Sex": "male",
      ... }
  ```  
- `labels=None`  
  這是用來存放「標籤」(例如`Survived`)，  
  預設是`None`，代表測試資料可能沒有標籤。這樣設計能通用於train/test兩種資料。  
  
- `features["Name"]`  
  代表從features中取出`Name`欄位的值，可能是一個句子，例如`"Smith, Mrs. Emma"`。  

- `tf.strings.split(...)`  
  這是TensorFlow的字串函式，會**把字串依照空格自動切成多個片段**(tokens)：  
  ```  
  tf.strings.split("Smith, Mrs. Emma")  
  → <tf.Tensor: shape=(3,), values=['Smith', 'Mrs.', 'Emma']  
  ```  

#### 第一段整行的意思: 
用空白切割每個名字，把原本的`"Smith, Mrs. Emma"`變成一個文字清單(tokens)，並**回寫進features["Name"]**。  

### 第二段: 建立`train_ds`訓練資料集
 
- `tfdf.keras.pd_dataframe_to_tf_dataset(...)`  
  這是TF-DF的API，作用是把Pandas的`DataFrame`轉成TensorFlow的`tf.data.Dataset`，這樣才能給模型用。  
- `preprocessed_train_df`  
  這是前面處理過的訓練資料(有做過`.apply(...)`)等前處理)。  
- `label="Survived"`  
  告訴TF-DF: 這份資料的「標籤」欄位是`Survived`，模型會以這個欄位作為預測目標。  
- `.map(tokenized_names)`  
  這是TensorFlow的Dataset方法，會讓每一筆資料都經過`tokenize_names`函數處理一次 → 做名字切割。  

### 第三段: 建立`serving_ds`訓練資料集
- `preprocessed_serving_df`  
  這是處理後的測試資料 (沒有`Survived`欄位)。  
- **沒有label**  
  因為我們沒有`Survived`可以提供，所以`label`參數不重要。  
- `.map(tokenize_names`  
  一樣對`Name`欄位進行分詞處理。

### 總結邏輯流程:
|步驟|說明|
|:-|:-|
|`tokenize_names()`|處理每一筆資料，把Name拆成token|
|`.map(...)`|套用到整個Dataset|
|`pd_dataframe_to_tf_dataset(...)`|把Pandas資料匡轉成TF-DF能用的Dataset格式|
|`train_ds / serving_ds`|建立兩個TensorFlow Dataset，後面會傳給模型訓練與預測使用|

## Train model with default parameters

In [6]:
# 第1部分: 建立Gradient Boosted Trees模型
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,    # Very few logs (關閉大部分訓練過程的訊息，畫面會比較乾淨)
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,   #Only use the features in "features" (只使用指定的欄位)
    random_seed=1234,
)

# 第2部分: 訓練模型與評估表現
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

# Accuracy: 模型預測對的比例 (準確率)
# Loss: 損失函數的數值，越小代表預測越準

W0000 00:00:1753139025.609280 5454667 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753139025.609408 5454667 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753139025.609410 5454667 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1753139026.928638 5454667 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1753139026.928673 5454667 kernel.cc:783] Collect training examples
I0000 00:00:1753139026.928680 5454667 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

Accuracy: 0.804347813129425 Loss:0.8922085165977478


### 語法說明:
|問題|是什麼|用來做什麼|
|:-|:-|:-|
|1. `GradientBoostedTreesModel()`|模型類型|建立GBT|
|2. `FeatureUsage()`|特徵指定物件|告訴模型要用哪些欄位|
|3. `exclude_non_specified_fetures`|選擇性參數|是否排除未指定的欄位|
|4. `random_seed`|隨機種子|讓結果可重現|
|5. `make_inspector().evaluation()`|模型評估|查看準確率與損失值|
|6. `.accuracy`/`.loss`|評估指標|評估訓練表現|

## Train model with improved default parameters

In [12]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,    #Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,   # Only use the features in "features"

    #num_trees=2000,

    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,

    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",

    #num_trees=1000,
    #tuner=tuner

    min_examples=1,
    categorical_algorithm="RANDOM",
    #max_depth=4,
    shrinkage=0.05,
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,

)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

W0000 00:00:1753079263.828805 4820429 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753079263.828833 4820429 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753079263.828838 4820429 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1753079263.932290 4820429 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1753079263.932302 4820429 kernel.cc:783] Collect training examples
I0000 00:00:1753079263.932306 4820429 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

Accuracy: 0.77173912525177 Loss:1.0119096040725708


I0000 00:00:1753079264.116079 4830559 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 1.01191
I0000 00:00:1753079264.116093 4830559 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 83
I0000 00:00:1753079264.117756 4830559 gradient_boosted_trees.cc:279] Truncates the model to 54 tree(s) i.e. 54  iteration(s).
I0000 00:00:1753079264.117850 4830559 gradient_boosted_trees.cc:341] Final model num-trees:54 valid-loss:1.011910 valid-accuracy:0.771739
I0000 00:00:1753079264.118425 4830559 kernel.cc:926] Export model in log directory: /var/folders/np/szgrlcss5093pf9ss6xq_mxh0000gn/T/tmpm7d7ua4p with prefix 19c859195e944d58
I0000 00:00:1753079264.119427 4830559 kernel.cc:944] Save model in resources
I0000 00:00:1753079264.119919 4820429 abstract_model.cc:921] Model self evaluation:
Task: CLASSIFICATION
Label: __LABEL
Loss (BINOMIAL_LOG_LIKELIHOOD): 1.01191

Accuracy: 0.771739  CI95[W][0 1]
Err

In [13]:
model.summary()

Model: "gradient_boosted_trees_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.431313 ################
    2.        "Pclass"  0.401411 ##############
    3.           "Age"  0.373986 ############
    4.          "Fare"  0.361093 ###########
    5.         "SibSp"  0.292547 #######
    6.         "Parch"  0.255448 ####
    7.          "Name"  0.225272 ##
    8.   "Ticket_item"  0.182111 
    9.      "Embarked"  0.181387 
   10. "Ticket_number"  0.180897 

Variable Importance: NUM_

### 小結: 這段summary的用途是?
|想了解什麼|看哪裡?|
|:-|:-|
|使用哪些欄位?|Input Features|
|哪些欄位最重要?|Variable Importance|
|模型有幾棵樹?深度?|Number of trees、Depth by leafs|
|訓練過程的準確率、損失|Training logs|

## Make predictions

In [16]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    import os
    if os.path.exists("/kagle/working"):
        path="/kaggle/working/submission.csv"
    else:
        path="../output/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head ../output/submission.csv

Submission exported to ../output/submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1


## Training a model with hyperparameter tunning

In [18]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss: {tuned_self_evaluation.loss}")

Use /var/folders/np/szgrlcss5093pf9ss6xq_mxh0000gn/T/tmpijooj08g as temporary training directory


W0000 00:00:1753081864.067130 4820429 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753081864.067168 4820429 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753081864.067170 4820429 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1753081864.167489 4820429 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1753081864.167506 4820429 kernel.cc:783] Collect training examples
I0000 00:00:1753081864.167512 4820429 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false


Accuracy: 0.8767123222351074 Loss: 0.6808041334152222


## Making an ensemble

In [21]:
predictions = None
num_predictions = 0

for i in range(100):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0,   # Very few logs
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True,  # Only use the features in "features"

        #min_examples=1,
        #categorical_algorithm="RANDOM",
        ##max_depth=4
        #shrinkage=0.05,
        ##num_candidate_attributes_ratio=0.2,
        #split_axis="SPARSE_OBLIQUE",
        #sparse_oblique_normalization="MIN_MAX",
        #sparse_oblique_num_projections_exponent=2.0,
        #num_trees=2000,
        ##validation_ratio=0.0
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)

    sub_predictions = model.predict(serving_ds, verbose=0) [:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)

import os
if os.path.exists("/kaggle/working"):
    path = "/kaggle/working/submission.csv"
else:
    path = "../output/submission.csv"
kaggle_predictions.to_csv(path, index=False)

i:0


W0000 00:00:1753083206.904334 4820429 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753083206.904364 4820429 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753083206.904366 4820429 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1753083207.051063 4820429 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1753083207.051077 4820429 kernel.cc:783] Collect training examples
I0000 00:00:1753083207.051081 4820429 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

i:1


I0000 00:00:1753083207.429066 5109813 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.729064
I0000 00:00:1753083207.429083 5109813 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 110
I0000 00:00:1753083207.432845 5109813 gradient_boosted_trees.cc:279] Truncates the model to 81 tree(s) i.e. 81  iteration(s).
I0000 00:00:1753083207.432908 5109813 gradient_boosted_trees.cc:341] Final model num-trees:81 valid-loss:0.729064 valid-accuracy:0.842593
I0000 00:00:1753083207.433773 5109813 kernel.cc:926] Export model in log directory: /var/folders/np/szgrlcss5093pf9ss6xq_mxh0000gn/T/tmpwb5tr66e with prefix e13ebd740ec24c3c
I0000 00:00:1753083207.435108 5109813 kernel.cc:944] Save model in resources
I0000 00:00:1753083207.435653 4820429 abstract_model.cc:921] Model self evaluation:
Task: CLASSIFICATION
Label: __LABEL
Loss (BINOMIAL_LOG_LIKELIHOOD): 0.729064

Accuracy: 0.842593  CI95[W][0 1]


i:2


...

W0000 00:00:1753083235.169395 4820429 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753083235.169404 4820429 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753083235.169405 4820429 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1753083235.251937 4820429 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1753083235.251950 4820429 kernel.cc:783] Collect training examples
I0000 00:00:1753083235.251954 4820429 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

i:98


W0000 00:00:1753083235.430261 4820429 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753083235.430270 4820429 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1753083235.430271 4820429 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
I0000 00:00:1753083235.512778 4820429 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1753083235.512787 4820429 kernel.cc:783] Collect training examples
I0000 00:00:1753083235.512790 4820429 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
column_guides {
  column_name_pattern: "^Pclass$"
}
column_guides {
  column_name_pattern: "^Name$"
}
column_guides {
  column_name_pattern: "^Sex$"
}
column_guides {
  column_name_pattern: "^Age$"
}
column_guide

i:99
Submission exported to ../output/submission.csv


I0000 00:00:1753083236.179552 5218648 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.751876
I0000 00:00:1753083236.179564 5218648 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 260
I0000 00:00:1753083236.182202 5218648 gradient_boosted_trees.cc:279] Truncates the model to 231 tree(s) i.e. 231  iteration(s).
I0000 00:00:1753083236.182266 5218648 gradient_boosted_trees.cc:341] Final model num-trees:231 valid-loss:0.751876 valid-accuracy:0.844444
I0000 00:00:1753083236.183280 5218648 kernel.cc:926] Export model in log directory: /var/folders/np/szgrlcss5093pf9ss6xq_mxh0000gn/T/tmp0p0jrmpv with prefix 8f2a5910e8db44c8
I0000 00:00:1753083236.185536 5218648 kernel.cc:944] Save model in resources
I0000 00:00:1753083236.186059 4820429 abstract_model.cc:921] Model self evaluation:
Task: CLASSIFICATION
Label: __LABEL
Loss (BINOMIAL_LOG_LIKELIHOOD): 0.751876

Accuracy: 0.844444  CI95[W][0 

In [22]:
evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {evaluation.accuracy}")
print(f"Loss: {evaluation.loss}")


Accuracy: 0.8767123222351074
Loss: 0.6808041334152222
